In [ ]:
import plotly.graph_objects as go
import pandas as pd
from pandas import DataFrame as df
import numpy as np
import plotly.express as px
from EC_data_processing_lib import get_header_length, get_decimal_separator
from Data_setup_CHEM import data_set  # choose the correct python file
import os
from galvani import BioLogic
from datetime import datetime

fig_title = 'Title'
today_str = datetime.today().strftime('%Y-%m-%d')
fig = go.Figure()
colors = px.colors.qualitative.Dark24

for i in data_set:
    file = data_set[i]['data_CA']
    if not os.path.exists(file):
        print("File not found: " + file)
        continue
    
    # Read the file based on its extension
    if file.endswith('mpt'):
        headerlength = get_header_length(file)
        data = pd.read_csv(
            file,
            encoding='ansi',
            sep="\t",
            decimal=get_decimal_separator(file),
            skiprows=range(headerlength)
        )
    else:
        mpr_file = BioLogic.MPRfile(file)
        data = pd.DataFrame(mpr_file.data)
    
    # Automatically select the correct current column
    if '<I>/mA' in data.columns:
        current_column = '<I>/mA'
    elif 'I/mA' in data.columns:
        current_column = 'I/mA'
    else:
        print("No valid current column found in file: " + file)
        continue

    # Normalize current by mass and get time (in seconds)
    current = data[current_column]
    time = data['time/s']
    
    fig.add_trace(go.Scatter(
        x=time,
        y=current,
        name=data_set[i]['label'],
        line_color=colors[data_set[i]['color_index']],
        line_width=3,
    ))

# Update axes styling with autoscaling
fig.update_xaxes(
    autorange=True,
    nticks=10,
    ticks='outside',
    showgrid=False,
    gridwidth=1,
    gridcolor='grey',
    zeroline=False,
    zerolinecolor='grey',
    zerolinewidth=1,
    showline=True,
    linewidth=2,
    linecolor='black'
)

fig.update_yaxes(
    autorange=True,
    nticks=10,
    showgrid=False,
    gridwidth=1,
    gridcolor='grey',
    ticks='outside',
    zeroline=True,
    zerolinecolor='grey',
    zerolinewidth=1,
    showline=True,
    linewidth=2,
    linecolor='black'
)

fig.update_layout(
    xaxis_title="Time (min)",
    yaxis_title="Current (mA/cm<sup>2</sup>)",
    font=dict(
        family="Times New Roman, monospace",
        size=18,
        color="dimgrey"
    )
)

# Set layout for SVG saving: transparent background
fig.update_layout({
    "plot_bgcolor": "rgba(0, 0, 0, 0)",
    "paper_bgcolor": "rgba(0, 0, 0, 0)"
},
    width=750,
    height=500,
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=1
    ),
    margin={'t': 0, 'l': 0, 'b': 0, 'r': 0},
    legend_font_size=10
)

# Save the SVG file first (transparent background)
fig.write_image(f"{today_str} {fig_title} Chronoamperometry.svg")

# Update layout for a white background for display and JPEG saving
fig.update_layout({
    "plot_bgcolor": "rgba(0, 0, 0, 0)",
    "paper_bgcolor": "rgba(255, 255, 255, 255)"
})

fig.show()

# Save the JPEG file with a white background
fig.write_image(f"{today_str} {fig_title} Chronoamperometry.jpeg")
